## ResNet Original Training & Inference

#### Training ResNet50 with Cifar 10

In [2]:
from ResNet.resnet import *
from ResNet.train import *

resnet_train_cifar10()

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/100 [05:21<?, ?it/s]


KeyboardInterrupt: 